# Log for project in Computer Science

By Anna and Eric

## Day 1, 4/4 - 2019

Today the goal is to understand the project and be prepared for mondays meeting.

Step 

     1) Download UniProt and check out the dataset

     2) Create the dictionary
     
     3) Get tool from Marcus
     
     4) Train a tagger with machine learning
     
So we would like to finish step 1 and start reseach on 2.

---
Today we sat from 13-16. We started by creating a private git repo with this file in it. Then we downloaded the uniprot dataset in xml form. We proceded to try and parse it with BeautifulSoup. The file was too big for that method.

After that we tried with ElementTree instead which can go through each element one by one. This worked and we eventually got a list of proteins (over 200 000).

We got stuck at which dictionary we want to create. Do we want one that's protein -> gene1 gene2 gene3 and thus also mine genes from the xml file. Or is the dictionary something else? For next time this needs to be cleared up in order to move on.

In [ ]:
#Goes through the file uniprot_sprot.xml and saves the names of the proteins
#in protein_names.txt
from xml.etree import ElementTree as ET
filename='uniprot_sprot.xml'
parser = ET.iterparse(filename)
length = 28 #length of header in xml tags
proteins = set()
for event, element in parser:
    tag = element.tag[length:]
    if 'shortName' in tag: # or 'shortName' in tag #fullName
        proteins.add(element.text)
    element.clear()
f = open('protein_names_short.txt', 'w')
f.write('\n'.join(proteins))
f.close()


## Day 2, 8/4 - 2019

Today we had our second meeting with Pierre and Sonja. We got clarification about the biology bit of the project but also some tips going forward. Now we have a Slack and are to get an invite to the Github repo. Also, an invite to Endnote.

TODO: Invite Sonja, Pierre to Overleaf document

Tips:

    1) Search PubMed for 'biomedical text-mining' and 'bio NLP' (Jensen)
    
    2) Get annotation from Hannes

So this week we wish to get the infrastructure up and running.

---
We started by looking at Marcus email and getting his program to work. After a lot of problems with installation of the various packages and programs we finally got it to work, almost 2h later.

Question: What is the file 'disease_index.fst' and how is it created? Should we create this?

---
We now have the basic infrastructure up and running. We have done some tests on running it with our dataset of proteins from UniProt. Also we started looking at BioCreative's datasets, but we'll leave that for next time. 


### Instructions to start JVM (need Marcus repo):

    1) cd [katalog]/mention-index-py4j

    2) mvn package

    3) cd target

    4) java -jar mentions-index-py4j-1.0-SNAPSHOT.jar

In [ ]:
# Install a conda package in the current Jupyter kernel
#import sys
#!{sys.executable} -m pip install docria
#!{sys.executable} -m pip install py4j
from docria import Document, MsgpackCodec, DocumentIO, set_large_screen, T
import os.path
from docria.printout import options
from docria.algorithm import group_by_span, dominant_right_span, dominant_right
set_large_screen()
def connect_jvm(port):
    from py4j.java_gateway import GatewayParameters, JavaGateway
    gateway_parameters=GatewayParameters(port=port, auto_convert=True, auto_field=True)
    gateway = JavaGateway(gateway_parameters=gateway_parameters)
    app = gateway.entry_point
    return gateway, gateway.jvm, app
gateway, jvm, app = connect_jvm(6006)
def get_java_file(path):
    # Here we do the py4j equivalent for new java.io.File(path)
    return jvm.java.io.File(os.path.abspath(path))

In [99]:
app.buildIndex(get_java_file("protein_names.txt"), get_java_file("protein_names.fst"))
indx = app.loadIndex(get_java_file("protein_names.fst"))

doc = Document()
doc.add_text("main","Comparison with alkaline phosphatases and 5-nucleotidase")
binary_doc = MsgpackCodec.encode(doc) 
search_binary_doc = app.search(indx, binary_doc)
doc = MsgpackCodec.decode(search_binary_doc)

tuls = []
for term in doc["matches"]:
    tuls.append((term["text"].start, term["text"].stop, str(term["text"])))
    #print(str(term["text"]), "-", "%d:%d" % (term["text"].start, term["text"].stop))
print(tuls)
dominant_right(tuls)


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:6006)

## Day 3, 9/4-2019
We have had some questions about the purpose and think we have finally figured it out. To identify the entities 'protein' and 'diseases' like cell death. To start with we will identify proteins by checking a pubmed article and using our uniprot list.



In [ ]:
#Goes through the file pubmed19n0651.xml and saves the abstract text as one textfile
from xml.etree import ElementTree as ET
filename='pubmed19n0651.xml'
out_name = 'abstract_text19n0651.txt'
parser = ET.iterparse(filename)
f=open(out_name, "a+", encoding="utf-8")
for event, element in parser:
    if element.tag == 'AbstractText':
        if element.text:
            f.write(element.text)
    element.clear()
f.close()

In [ ]:
from docria import Document, MsgpackCodec, DocumentIO, set_large_screen, T
import os.path
from docria.printout import options
set_large_screen()
def connect_jvm(port):
    from py4j.java_gateway import GatewayParameters, JavaGateway
    gateway_parameters=GatewayParameters(port=port, auto_convert=True, auto_field=True)
    gateway = JavaGateway(gateway_parameters=gateway_parameters)
    app = gateway.entry_point
    return gateway, gateway.jvm, app
gateway, jvm, app = connect_jvm(6006)
def get_java_file(path):
    return jvm.java.io.File(os.path.abspath(path))

In [ ]:
#Prints all occurences of proteins in the abstract file.
app.buildIndex(get_java_file("protein_names_long.txt"), get_java_file("protein_names_long.fst"))
indx = app.loadIndex(get_java_file("protein_names_long.fst"))
data = ''
with open('abstract_text19n0651.txt', 'r', encoding="utf-8", errors='ignore') as file:
    data = file.read()
sp = data.split('.')
for s in sp:
    hits = indx.search(s)
    for hit in hits:
        print(hit.start, hit.end, " ".join([term.raw for term in hit.terms]))

In [ ]:
#Prints all occurences of cell deaths in the abstract file.
app.buildIndex(get_java_file("cell_death_names.txt"), get_java_file("cell_name_index.fst"))
indx = app.loadIndex(get_java_file("cell_name_index.fst"))
data = ''
with open('abstract_text19n0651.txt', 'r', encoding="utf-8", errors='ignore') as file:
    data = file.read()
sp = data.split('.')
for s in sp:
    hits = indx.search(s)
    for hit in hits:
        print(hit.start, hit.end, " ".join([term.raw for term in hit.terms]))

Next we tried to use the model that https://aclweb.org/anthology/W16-5104 implemented at github: https://github.com/withtwist/medical-ner. After transpiling the code to Python 2, changing the encoding of the data etc. we still got compilation errors. It seemed too big of a job so we scraped it. Since we have been able to identify proteins from the abstract and where they are located, we are now interested in finding a way to create the tracer with machine learning. We found the article *Unsupervised biomedical named entity recognition: Experiments with clinical and biological texts. 


Note: We noticed that the indx.search is case sensitive, which needs to be fixed. (e.g. toLowerCase or similar). Check w Marcus!!

Conclusion of todays work: We have fixed the dictionaries and spent some time making sure the xml reader for uniprot included the edgecases and included 'shortName'. We also started tinkering with neural networks by downloading existing repos but we could not make it work. We started reading a couple of papers to get inspiration for our tagger. 

For next time, we will look deeper into the neural networks and use the tips Marcus sent us.

## Day 4, 14/4-2019
We prepared for tomorrows meeting by looking at Marcus tips that he sent us. Also we started looking at more journals and papers and found a really good one: [A Novel Approach for Protein-Named Entity Recognition and Protein-Protein Interaction Extraction](https://www.hindawi.com/journals/mpe/2015/942435/). They describe step-by-step a method with SVM to extract the entities. If they agree on tomorrows meeting we will follow this guide next week.

## Day 5, 15/4-2019
Today after the meeting we have a clearer understanding how to move forward. The process follows:
    1. Add list of corpus with summarization of each content and size to github
    2. [x] Fix java to ignore case
    3. [x] Complete a working model with the dictionary model
        i. [x] ~~Fix reader and use Element tree instead.~~ Also read in 'pmid' and change output to 'pmid \t abstract_text \n'.
        ii. [x] Match output to corpus annotation
    4. [ ] Create evaluator for dictionary model
        i. Use [genetag](https://www.ncbi.nlm.nih.gov/pubmed/15960837) for evaluation or chemner? or both?
        ii. Compare with e.g recall
    5. Add more to dictionary and run evaluator to see if it improves. If ambigueties use dominant write.
    
After this is completed we will start on the machine learning model.
    1. Use POS? Pre-processing?
    2. Sentence -> Tokens -> Tagger -> Phrases -> ML (BioLSTM and Bioascembedding(spelling?))
    3. Check out page [nlpprogress](http://nlpprogress.com/english/named_entity_recognition.html) for NER datasets
   

Möte med Sonja: https://biocreative.bioinformatics.udel.edu/resources/corpora/biocreative-iii-corpus/

---

In [ ]:
indx = app.loadIndex(get_java_file("protein_names_long.fst"))

data = ''
with open('genetag.db', 'r', encoding="utf-8", errors='ignore') as file:
    data = file.read()
sp = data.split('\n>>')
dictionary = {}
for s in sp:
    lines = s.split('\n')
    #If it is an excerpt
    if lines[0] == 'EXCERPT':
        id = ''
        text = ''
        for line in lines[1:]:
            if line.startswith('EXCERPT_ID'):
                id = line.split(': ')[1]
            elif line.startswith('TEXT'):
                text = ''.join(line.split(': ')[1:])
        dictionary[id] = text

In [ ]:
# use indx.search to find all occurrences of proteins in abstracts from genetag. 
indx = app.loadIndex(get_java_file("protein_name_index.fst"))

d = dictionary
out = []
for key in d:
    value = d[key]
    hits = indx.search(value)    
    for hit in hits:
        # term.raw for term in hit.terms prints lists of protein names. Each list is protein name
        out.append('|'.join([key, str(hit.start) + ' ' + str(hit.end), " ".join([term.raw for term in hit.terms])]))
file = open('genetag.out', 'w+', encoding="utf-8", errors='ignore')
file.write('\n'.join(out))
file.close()
    

---
Today we sat for quite some time with merge conflicts in jupyter notebooks as it would not compile after a while. Also we tried to implement 2 models for evaluation; genetag with perl and another one with python 2.5. The gentag could have worked, but we got such a low score that it probably checks line by line simmilarity or something like it. This means we only got 8 correct matches which seems very low. We will look further into this next time.

We succesfully changed the tokenizers in java by removing 2 of them and adding one to ignore cases. We became aware of how many small words were in the 'shortName' tag from chemprot. So we got protein names like 'AM, OR, AND' etc. We thus created a textfile 'protein_names_long' which contains only the long names. This meant we got a more expected result, but we are also missing out on some of the real names. TODO: Think about how to include the short names - do we want to set a limit of min 2 characters?

Finally we created an out file to match the evaluation file in genetag. It reads from the database and using our proteinnames builds the same style file.

## Day 6, 17/4-2019
We started by creating another file 'protein_names_short', and discussed whether to use this in combination with long names or only long names. For now we will only use long.

Today's goal is to create/find/adapt an evaluator that works on our data.

We can run the perl evaluator from genetag and get the following result:
```
TP: 2
FP: 18211
FN: 406
Precision: 0.000109811672980838
Recall: 0.00490196078431373
```
Which is very bad. we think this is because the java tokenizer changes words like "h-55" to "h - 55". So it matches both 'h' and '55'. We need to change such that the hyphen does not get split in this way. There are probably more errors as well.


In [38]:
from docria import Document, MsgpackCodec, DocumentIO, set_large_screen, T
import os.path
from docria.algorithm import group_by_span, dominant_right_span, dominant_right
from docria.printout import options
set_large_screen()
def connect_jvm(port):
    from py4j.java_gateway import GatewayParameters, JavaGateway
    gateway_parameters=GatewayParameters(port=port, auto_convert=True, auto_field=True)
    gateway = JavaGateway(gateway_parameters=gateway_parameters)
    app = gateway.entry_point
    return gateway, gateway.jvm, app
gateway, jvm, app = connect_jvm(6006)
def get_java_file(path):
    return jvm.java.io.File(os.path.abspath(path))

p_type = 'long'

app.buildIndex(get_java_file("protein_name/protein_names_{}.txt".format(p_type)), get_java_file("protein_name/protein_names_{}.fst".format(p_type)))
indx = app.loadIndex(get_java_file("protein_name/protein_names_{}.fst".format(p_type)))


#READ INPUT
data = ''
proteins = set()
with open('genetag.db', 'r', encoding="utf-8", errors='ignore') as file:
    data = file.read()
sp = data.split('\n>>')
dictionary = {}

for s in sp:
    lines = s.split('\n')
    #If it is an excerpt
    if lines[0] == 'EXCERPT':
        id = ''
        text = ''
        for line in lines[1:]:
            if line.startswith('EXCERPT_ID'):
                id = line.split(': ')[1]
            elif line.startswith('TEXT'):
                text = ':'.join(line.split(':')[1:])[1:]
        dictionary[id] = text
    elif lines[0] == 'ANNOTATION':
        id = ''
        text = ''
        for line in lines[1:]:
            if line.startswith('TEXT:'):
                text = ':'.join(line.split(':')[1:])[1:]
        proteins.add(text)
print(len(proteins))
with open('protein_name/protein_names_{}.txt'.format(p_type), 'a', encoding='utf-8', errors='ignore') as file:
    file.write('\n'.join(proteins))
file.close()

app.buildIndex(get_java_file("protein_name/protein_names_{}.txt".format(p_type)), get_java_file("protein_name/protein_names_{}.fst".format(p_type)))
indx = app.loadIndex(get_java_file("protein_name/protein_names_{}.fst".format(p_type)))

22315


In [47]:
import re
d = dictionary
out = []

for key in d:
    doc = Document()
    doc.add_text("main", d[key])    
    binary_doc = MsgpackCodec.encode(doc) 
    search_binary_doc = app.search(indx, binary_doc)
    doc = MsgpackCodec.decode(search_binary_doc)  

    tuls = []
    for term in doc["matches"]:
        tuls.append((term["text"].start, term["text"].stop, str(term["text"])))
    print(tuls)    
    matches = [(x,y,z) for (x,y,z) in tuls if z in dominant_right(tuls)]
    
    for m in matches:
        #if re.search('(?!^\d+$)^.+$', m[2]): #removes pure numbers
            out.append('|'.join([key, str(m[0]) + ' ' + str(m[1]), m[2]]))
        
file = open('genetag/genetag_{}.out'.format(p_type), 'w+', encoding="utf-8", errors='ignore')
file.write('\n'.join(out))
file.close()

[]
[]
[]
[(37, 43, 'HLA-B5')]
[]
[]
[(11, 27, 'immunoglobulin E')]
[(42, 64, 'chloroacetate esterase')]
[(145, 151, 'fibrin'), (181, 191, 'FR-antigen'), (277, 289, 'cryoglobulin'), (300, 317, 'plasma fibrinogen')]
[]
[]
[]
[(77, 94, 'factor IX antigen')]
[(21, 42, 'alpha 2-macroglobulin'), (44, 63, 'alpha 1-antitrypsin'), (65, 76, 'plasminogen'), (78, 91, 'C3-complement'), (93, 130, 'fibrinogen degradation products (FDP)')]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(105, 134, '1.8 kb HindIII-EcoRI fragment')]
[]
[]
[]
[]
[]
[]
[]
[]
[(31, 49, 'cytochrome oxidase'), (51, 61, 'peroxidase')]
[]
[]
[(70, 100, 'cytochrome P-450 monooxygenase')]
[]
[(30, 37, 'insulin')]
[]
[]
[(137, 138, 'T'), (185, 208, 'X. laevis oocyte 5S DNA')]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(181, 184, 'IgA')]
[]
[]
[]
[(10, 20, 'hemoglobin'), (102, 107, 'white')]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(12, 17, 'CK-MB'), (36, 50, 'myocardial CPK

[(50, 65, 'Fos-Jun complex'), (72, 81, 'AP-1 site'), (89, 119, 'type alpha I collagen promoter')]
[]
[]
[]
[(9, 12, 'CBF'), (29, 32, 'CBF'), (60, 61, 'T')]
[]
[]
[(158, 225, 'murine class II major histocompatibility complex A alpha gene X box')]
[]
[(52, 56, 'AHA2'), (136, 137, 'C')]
[]
[(4, 12, 'nmr gene')]
[]
[(130, 132, 'CT'), (270, 271, 'L'), (324, 325, 'L')]
[]
[(113, 131, 'beta-galactosidase'), (172, 200, 'polyomavirus enhancer region')]
[(19, 28, 'T antigen'), (48, 63, 'topoisomerase I'), (133, 135, 'T-')]
[(55, 73, '7.5-kb COR segment')]
[]
[]
[]
[]
[(20, 33, 'O7-LPS region'), (127, 135, 'Tn3HoHo1'), (166, 176, 'lac operon'), (204, 232, 'lacZ transcriptional fusions')]
[(4, 13, 'sacT gene'), (29, 62, 'sacPA operon in Bacillus subtilis')]
[(18, 27, 'IRA2 gene'), (177, 189, 'cdc25 mutant')]
[]
[]
[(185, 202, 'Fur binding sites')]
[(36, 54, 'beta A-globin gene')]
[(28, 32, 'NF-Y'), (37, 45, 'CBF DNAs')]
[]
[]
[(34, 61, 'variant tRNA(Val)(UAC) gene')]
[(66, 110, 'chloramphenicol ac

[]
[]
[]
[]
[]
[(45, 77, 'M RNA product of Uukuniemi virus')]
[]
[]
[]
[]
[]
[]
[(99, 100, 'E')]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(139, 140, 'P')]
[]
[]
[]
[]
[]
[]
[(82, 83, 'P'), (115, 118, 'max'), (120, 121, 'r')]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(118, 138, 'E/S and CWE antigens')]
[]
[]
[]
[(10, 47, 'Chinese hamster S14 protein sequences')]
[]
[]
[]
[(117, 129, 'ToxR protein'), (161, 162, 'N')]
[]
[(146, 184, 'plasma glutathione peroxidase (GSH-Px)')]
[(519, 522, 'max')]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(207, 208, 'P'), (226, 228, 'SR'), (239, 241, 'SR')]
[]
[]
[]
[(32, 45, 'CSF prolactin')]
[]
[]
[(70, 88, 'sry messenger RNAs')]
[]
[(72, 83, 'HS promoter'), (91, 115, 'soybean HS-protein genes')]
[]
[(92, 94, 'O2'), (99, 119, 'oxyhemoglobin (HbO2)')]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[(118, 119, 'M')]
[(44, 59, 'HDL-cholesterol'), (83, 99, 'HDL2-cholesterol'), (135, 162, 'hepatic triglyceride lipase')]
[]
[]
[]
[]
[]
[]
[]


[(138, 148, 'CAT fusion')]
[(49, 51, 'E2')]
[(157, 177, 'E1 recognition motif')]
[]
[]
[(27, 54, 'sGTH alpha subunit promoter'), (96, 110, 'sGTH alpha/CAT'), (215, 218, 'GSE')]
[]
[(4, 10, 'Myc LZ'), (74, 77, 'Myc')]
[(13, 35, 'active FecR derivative')]
[(15, 24, 'psbB-psbT'), (29, 38, 'psbH-psbN')]
[]
[(103, 136, 'bovine liver eIF-2B alpha subunit')]
[]
[(19, 24, 'PRP21'), (38, 43, 'SPP91'), (46, 50, 'CDC6'), (52, 56, 'CRY2'), (80, 101, 'ribosomal protein S24')]
[]
[(59, 75, 'GH receptor gene'), (151, 157, 'distal'), (264, 266, 'P1')]
[(21, 39, 'MEF2 reporter gene'), (102, 111, 'MEF2 site')]
[(63, 68, 'MKP-1')]
[(6, 10, 'AP-1')]
[]
[(82, 92, 'hprt locus')]
[]
[(135, 146, 'cotJ operon'), (212, 216, 'coat'), (264, 268, 'coat')]
[]
[(99, 102, 'ORF'), (192, 232, 'Neurospora crassa copper metallothionein')]
[(36, 48, 'BRCA1 region'), (152, 153, 'T')]
[(76, 86, 'Ash-l cDNA')]
[]
[(13, 20, 'COUP-TF')]
[]
[]
[]
[(86, 95, 'RAR alpha'), (99, 102, 'PML')]
[]
[(24, 72, 'amphibian insulin receptor

[]
[]
[]
[(154, 156, 'U5')]
[(55, 74, 'DNA-binding protein'), (106, 120, 'c-fos promoter')]
[]
[(60, 90, 'gramicidin S synthetase 2 gene')]
[]
[]
[]
[]
[]
[]
[]
[]
[(72, 80, 'HDC mRNA')]
[(117, 122, 'CeGAP'), (180, 236, 'COOH-terminal segment (GTPase-activating protein region)'), (240, 243, 'Bcr'), (260, 286, 'GTPase-activating proteins'), (294, 307, 'Rho subfamily')]
[(107, 123, 'Oct-3/4 promoter'), (127, 130, 'P19')]
[(52, 60, 'COUP-TFs'), (107, 149, 'Oct4 embryonic stem cell-specific enhancer')]
[(41, 44, 'Myc')]
[(94, 108, '5-HT3 receptor')]
[]
[(41, 51, 'SAP-1 cDNA'), (64, 76, 'mature SAP-1'), (113, 135, 'transmembrane-type PTP'), (162, 182, 'PTP-conserved domain')]
[]
[(42, 86, 'CCAAT/enhancer binding protein (C/EBP) alpha'), (122, 158, 'rat uncoupling protein gene promoter')]
[]
[]
[]
[(93, 108, 'relaxase (TraI)'), (132, 135, 'RP4')]
[]
[]
[]
[]
[(57, 83, 'NTP-binding helicase motif'), (115, 135, 'RNA polymerase motif')]
[(15, 35, 'F3R late transcripts'), (55, 56, 'A')]
[(300, 3

...

[]
[]
[(31, 47, 'mutant recN gene')]
[]
[]
[(0, 31, 'Connector enhancer of KSR (CNK)'), (70, 73, 'RAS')]
[(46, 56, 'APS kinase')]
[(0, 57, 'Epstein-Barr virus (EBV) latent membrane protein 1 (LMP1)')]
[(43, 91, 'homolog of alphavirus nonstructural protein nsP1')]
[]
[(30, 50, 'F beta alpha/CG beta'), (91, 111, 'CG and FSH receptors')]
[]
[(106, 169, 'nuclear transcriptosomal component scaffold attachment factor B')]
[]
[(4, 12, 'MXR gene'), (182, 202, 'MXR half-transporter')]
[]
[(0, 42, 'Serum insulin-like growth factor I (IGF-I)')]
[]
[]
[(130, 146, 'tyrosine kinases')]
[]
[]
[]
[(33, 43, 'TK domains'), (92, 105, 'scavenger RTK'), (150, 171, 'TK class II receptors'), (220, 227, 'insulin'), (232, 254, 'insulin-like receptors')]
[(101, 118, 'HindIII fragments')]
[]
[(43, 81, 'Fanconi anemia group C protein (FANCC)')]
[]
[]
[]
[(64, 77, 'beta-arrestin'), (105, 108, 'Src'), (125, 133, 'clathrin'), (183, 187, 'GPCR'), (197, 200, 'ERK'), (251, 254, 'ERK')]
[(40, 61, '47-kDa P-CIP2 protein'

[]
[]
[]
[(86, 157, 'homeodomain transcription factor thyroid transcription factor-1 (TTF-1)')]
[]
[]
[]
[]
[(129, 180, 'human telomerase reverse transcriptase (hTERT) gene')]
[]
[(183, 202, 'U5 and U4/U6 snRNPs')]
[]
[(77, 86, 'mod(mdg4)'), (90, 111, 'Drosophila. mod(mdg4)'), (255, 280, 'gypsy insulator sequences'), (296, 309, 'homeotic gene')]
[(21, 42, 'AfsA homologue (BarX)'), (88, 97, 'regulator'), (183, 185, 'M1')]
[]
[]
[(70, 80, 'histone H1'), (153, 155, 'H1')]
[]
[]
[]
[(105, 110, 'Cdc42')]
[]
[(0, 24, 'Jkappa DNA-binding sites'), (70, 92, 'mutant EBNA-3C protein')]
[(83, 100, 'HIV-1 ENV epitope'), (116, 135, 'MHC class I antigen')]
[(67, 86, 'pleckstrin homology'), (91, 113, 'Src homology 2 domains'), (131, 132, 'T')]
[(154, 157, 'FAK')]
[(34, 53, 'yeast PHO8 promoter'), (87, 100, 'PHO5 promoter')]
[(31, 45, 'I-type lectins'), (47, 82, 'immunoglobulin superfamily proteins')]
[]
[(95, 99, 'DDP1')]
[(68, 102, 'eIF2Bepsilon (residues 518 to 712)'), (164, 168, 'eIF2')]
[(47, 68, 

[]
[(21, 43, 'alpha1,3GT transcripts')]
[(40, 44, 'RNA3')]
[(61, 65, 'GAIP')]
[]
[]
[(51, 60, 'CREMalpha'), (108, 154, 'phosphoenolpyruvate carboxykinase (PEPCK) gene')]
[(17, 26, 'HNF3 site'), (52, 59, 'NF1/CTF')]
[(65, 102, 'forkhead transcription factor FKHR-L1')]
[(63, 72, 'p27(KIP1)'), (95, 115, 'PI3K-induced FKHR-L1')]
[(51, 64, 'r-PTPeta gene'), (144, 149, 'v-mos'), (154, 172, 'v-ras-Ki oncogenes')]
[(98, 143, 'Raf-MEK-extracellular signal-regulated kinase')]
[]
[]
[]
[]
[(47, 49, 'A2'), (50, 52, 'A3')]
[]
[]
[]
[]
[(182, 238, 'type II regulatory subunit of protein kinase A (PKA RII)')]
[(165, 175, 'hemoglobin')]
[(84, 118, 'FAP-1 (C-terminal 399 amino acids)'), (165, 196, 'glutathione-S-transferase (GST)')]
[(114, 135, 'N-terminal BIR domain')]
[]
[(0, 31, 'Ang II-induced fibronectin mRNA'), (136, 166, 'dominant negative EGF-R mutant'), (212, 228, 'fibronectin mRNA')]
[]
[(29, 33, 'S6K1'), (41, 45, 'core'), (100, 104, 'S6K1'), (112, 113, 'N'), (119, 120, 'C'), (205, 206, 'C'), 

[]
[(12, 22, 'mCTR mRNAs'), (101, 114, 'mCTR isoforms')]
[(193, 197, 'GHBP'), (204, 260, 'membrane-associated GHR transmembrane/cytoplasmic domain')]
[]
[(24, 48, 'factor V Leiden mutation'), (94, 123, 'angiotensin converting enzyme')]
[]
[]
[(75, 100, 'rat eIF4E promoter region')]
[]
[]
[]
[]
[]
[(0, 14, 'SH2D1A protein'), (83, 98, 'B cell receptor')]
[(26, 33, 'wt-ITAM'), (37, 45, 'IIA-ITAM'), (73, 84, 'PI 3-kinase'), (109, 117, 'PKBalpha')]
[(116, 124, 'p53(-/-)'), (151, 159, 'p53 gene')]
[(50, 53, 'Crk'), (111, 114, 'Abl'), (119, 122, 'FAK'), (208, 240, 'SH3 linker/C-terminal SH3 domain')]
[]
[]
[(34, 62, 'RNA dependent RNA polymerase')]
[]
[(83, 105, 'full-length DENTT cDNA')]
[(44, 78, 'protein translocase component SecE'), (84, 111, 'antiterminator protein NusG'), (121, 150, 'ribosomal proteins L11 and L1'), (166, 170, 'PkwR'), (209, 225, 'LacI-GalR family')]
[(11, 20, 'FRAP/mTOR'), (40, 46, '4E-BP1')]
[]
[(125, 137, 'N-SH2 domain'), (184, 196, 'C-SH2 domain')]
[(4, 15, 'UBA dom

[]
[]
[]
[(56, 59, 'AFP')]
[]
[]
[]
[]
[]
[(25, 26, 'N'), (31, 32, 'N'), (33, 35, 'H3'), (38, 39, 'N'), (98, 99, 'L')]
[]
[]
[]
[]
[]
[]
[]
[]


---
We chatted with Marcus for quite a bit today. As we wanted to add dominant_right we first tried to do it in Java but after a lot of trial-and-error it did not work and we switched to the python version. We also had to switch the way wee send in data as we needed to use docria instead of python.

Additionally we started getting back to the filters in java that we originally had. We also tried to work on the evaluator, but we are suspecting that something is weird in it as we get an unexpected nbr of matches (only ~27 TruePositive). 

For next time: add stopwords and fix min_length of matches.

### To run the perl evaluator
- go to medtag/genetag
- run `perl alt_eval.perl ../[PATH]/genetag_short.out Gold.format > answer.txt`


## Day 7 6/5-2019
Today's meeting: For next week we want a functioning demo. Want something to show eventhough it is not that accurate. GUI not priority but Pierre would like to do everything with the push of a button.

Keep 2 layers, one fuzzy and remove the matches from fuzzy if it overlaps with the exact layer.

TODO: Fix the Gold.format file. Right now it does not have the correct offset and thus gives us the terrible score. Second, we need to play around with the filters and keep building on 2 different analyzers for the short and long words. Finally tweak so that we can present a solution next time.

---

In [98]:
#Fix gold format file
with open('../../medtag/genetag/Gold.format', 'r', encoding="utf-8", errors='ignore') as file:
    data = file.read()

prot = {}
fixed = {} 
for line in data.split('\n'):
    l = line.split('|')
    if len(l) > 2:
        id = l[0]
        [i1, i2] = l[1].split()
        p = l[2]
        prot.setdefault(id, []).append(p)

for key in dictionary:
    if key in prot:
        proteins = prot[key]
        start_i = 0
        previous = ''
        for p in proteins:
            if p == previous:
                start_i += len(previous)
            match_start = start_i + dictionary[key][start_i:].index(p)
            match_end = match_start + len(p)
            start_i = match_start
            fixed.setdefault(key, []).append('{}|{} {}|{}'.format(key, match_start, match_end, p))
            previous = p

elems = []
for key in sorted(fixed.keys()):
    for e in fixed[key]:
        elems.append(e)

with open('Gold2.format', 'w', encoding='utf-8', errors='ignore') as file:
    file.write('\n'.join(elems))


With this new Gold.format we got:

genetag_long: ```Precision: 0.517406105864265 Recall: 0.350740713169061```. Gives: F1 = 0.42

genetag_short: ```Precision: 0.11398451655411 Recall: 0.309527359475175```

We changed the filters for genetag short (only minimum length = 2 and split on whitespace) and recieved a better result:

genetag_short: ```Precision: 0.522418879056047 Recall: 0.291714709273596```. Gives: F1 = 0.37

Now we also need to find a way to combine genetag_short and long and score them together. One idea is to use dominant right for each abstract. Another is to simply smack them together. As the evaluator is not dependent on the order of the id's, we will try the simple solution of stacking them together first.

genetag: ```Precision: 0.288077969174977 Recall: 0.488497227255257```

When stacking them together we don't get TP(short)+TP(long), so there seems to be an overlap of ~4000 proteins. So we must use something like dominant right to fix this overlap.

In [ ]:
#Build together long and short


## Day 8 7/5-2019
We have consulted Marcus on what todo going forward with adding genetag_short and genetag_long. He said we have 2 options:
    1. Add them both to one layer and do Dominant Right
    2. Prioritize one detection method over the other, i.e. find all the intersections, remove the overlaps and then join the layers.

So for today, our todo is as follows:
    1. Remove numbers from the genetag lists (pure numbers like 555)
    2. Join genetag long and short using one of the methods above
    3. Evaluate and find possible improvements
    4. Add list of cell death synonyms
    5. Fix demo that we can show
    
---

We will first look at the filters in java to see if there is one ready to use. There wasn't one. We will then do some postprocessing and remove those matches. We did this ```if re.search('(?!^\d+$)^.+$', m[2]): #removes pure numbers``` which will suffice for now.

genetag_short: ```Precision: 0.523598384077249 Recall: 0.291769615110086``` Without the only number ones. So slightly improved (by ~3 decimals).

genetag_long: ```Precision: 0.336532053845129 Recall: 0.415911711414923``` So this got worse recall? What happened? 

(genetag_long: ```Precision: 0.336532053845129 Recall: 0.415911711414923``` F1 = 0.37. When we swiched ICUFoldingFilter to toLowerCaseFilter. So we'll go with ICU instead)

In [ ]:
#Remove numbers from the genetag lists

Trying different filters for genetag_long:

| Activated filter | Precision | Recall | F1 |
|----------------------------------------|-----------|--------|----|
| Mention, ICUNorm, Diacratic, Snowball  | 0.623715592889822 | 0.456596936254324 | 0.527 |
| Mention, ICUNorm, Diacratic, Snowball, ICUFolding | 0.350802099229538 | 0.517487508922198 | 0.418 |
| Mention, ICUNorm, Diacratic, Snowball, StopFilter | 0.615680473372781 | 0.457036182946247 | 0.524 |
| Mention, ICUNorm, Diacratic, Snowball, LengthFilter | 0.287200716043858 | 0.422829846812716 | 0.342 |
| Mention, ICUNorm, Diacratic, Snowball, toLowerCase | 0.350802099229538 | 0.517487508922198 | 0.418 |
| Mention, ICUNorm, Diacratic | 0.651159123166693 | 0.45341239773788 | 0.535 |
| Mention, ICUNorm |  0.651159123166693 | 0.45341239773788 | 0.535 |
| Mention |  0.651159123166693 | 0.45341239773788 | 0.535 |
| Mention (without removing numbers) |  0.651107782070488 | 0.45341239773788 | 0.535 |

So for genetag_long the best solution seemed to be to not really add anything.

---

Different filters for genetag_short:

| Activated filter | Precision | Recall | F1 |
|----------------------------------------|-----------|--------|----|
| PatternTokenizerFactory | 0.61393618220769 | 0.298962279690331 | 0.402 |
| Mention | 0.629489888318744 | 0.458024488003075 | 0.530 |
| Mention, LengthFilter | 0.324438202247191 | 0.418547191566464 | 0.366 |
| Mention (without removing numbers) | 0.629489888318744 | 0.458024488003075 | 0.530 |


In [52]:
#Join genetag long and short using one of the methods above
data_long = ''
data_short = ''

out = []

with open('genetag/genetag_long.out', 'r', encoding='utf-8', errors='ignore') as file:
    data_long = file.read().split('\n')
with open('genetag/genetag_short.out', 'r', encoding='utf-8', errors='ignore') as file:
    data_short = file.read().split('\n')

def read(data, dic):
    for line in data:
        tul = line.split('|')
        key = tul[0]
        line_nbr = tul[1].split()
        start = line_nbr[0]
        end = line_nbr[1]
        word = tul[2]
        dic.setdefault(key, []).append((int(start), int(end), word))
    return dic

dic = read(data_short, read(data_long, {}))

for key in dic:
    tuls = dic[key]
    matches = [(x,y,z) for (x,y,z) in tuls if z in dominant_right(tuls)]
    matches = list(set(matches))
    
    for m in matches:
        out.append('|'.join([key, str(m[0]) + ' ' + str(m[1]), m[2]]))
        
with open('genetag/genetag.out', 'w+', encoding="utf-8", errors='ignore') as file:
    file.write('\n'.join(out))

Combined genetag_long and genetag_short, i.e. genetag.out: ```0.592672838290329 Recall: 0.495634985999012```
F1 = 0.540